#### Working with SQLite : Introduction

SQLite is built in Python to provide a relational database management system. It is
very easy to set up, very fast and lightweight, and thus referred to as ‘lite’. Here are
some very important features to note about SQLite: self-contained, serverless, with
zero-configuration needed to run and transactional.

● Self-Contained

    This means that it does not need much support from the operating system
    or external libraries. This makes it suited for use in embedded devices like
    mobile phones, iPods and game devices that lack the infrastructure of a
    computer. The source code is found in files called sqlite3.c and the header
    file sqlite3.h. When you want to use SQLite in an application, ensure that you
    have these files in your project directory when compiling your code.

● Serverless

    In most cases, relational database management systems require a separate
    server to receive and respond to requests sent from the client.Such systems include MySQL and Java Database Client -JDBC. These clients
    have to use the TCP/IP protocol to send and receive responses. This is
    referred to as the Client/Server architecture. On the other hand, SQLite does
    not make use of a separate server. While using SQLite, the application reads
    and writes directly to the database files stored on the application server disk.

● Zero-configuration

    You don’t need to install SQLite prior to using it in an application or system.
    This is because of the serverless characteristic described previously.

● Transactional

    All transactions in SQLite are Atomic, Consistent, Isolated, and Durable
    (ACID- compliant). In other words, if a transaction occurs that attempts to
    make changes to the databases, the changes will either be made in all the
    appropriate places (in all linked tables and affected rows) or not at all. This is
    to ensure data integrity.

#### Python's SQLite module

It is easy to create and manipulate databases with Python. To allow us to use
SQLite with Python, the Python Standard Library includes a module called "sqlite3".
To use the SQLite3 module, we need to add an import statement to our Python
script:

In [2]:
import sqlite3

We can then use the function sqlite3.connect to connect to the database. We
pass the name of the database file to open or create it

In [3]:
# Creates or opens a file called student_db with a SQLite3 DB

db = sqlite3.connect('student_db')

# Sanity check, because I'm paranoid.
print('Connection Established!')

Connection Established!


#### Creating & deleting tables

To make any changes to the database, we need a cursor object. A cursor object is
an object that is used to execute SQL statements. Next, .commit is used to save
changes to the database. It is important to remember to commit changes since
this ensures the atomicity of the database. If you close the connection using close
or the connection to the file is lost, changes that have not been committed will be
lost.

Below we create a student table with id, name and grade columns.

In [4]:
cursor = db.cursor()

cursor.execute('''
    CREATE TABLE IF NOT EXISTS student(id INTEGER PRIMARY KEY, name TEXT,
                grade INTEGER)
''')

db.commit()

# Sanity check
print('Table Created!')

Table Created!


Always remember that the commit function is invoked on the db object, not the
cursor object. If we type cursor.commit, we will get the following error message:
“AttributeError: 'sqlite3.Cursor' object has no attribute 'commit'”

#### Inserting into the Database

To insert data, we use the cursor again to execute a SQL statement. When using
data stored in Python variables to insert data into a database table, use the "?"
placeholder. It is not secure to use string operations or concatenation to make your
queries.
In this example, we are going to insert two students into the database; their
information is stored in Python variables.


In [7]:
id1 = 1001
name1 = 'Jimbo'
grade1 = 78

id2 = 1002
name2 = 'John'
grade2 = 80

# insert first student
cursor.execute('''
    INSERT INTO student(id, name, grade)
                VALUES(?,?,?)
''', (id1,name1,grade1))

print('First user added!')

cursor.execute('''
    INSERT INTO student(id, name, grade)
                VALUES(?,?,?)
''', (id2,name2,grade2))

print('Second user added!')

db.commit()

First user added!
Second user added!


In the example below, the values of the Python variables are passed inside a tuple
(you could also use a dictionary):

In [8]:
id3 = 1003
name3 = 'Johnny'
grade3 = 68

cursor.execute('''
    INSERT INTO student(id, name, grade)
                VALUES(:id, :name, :grade)
''', {'id':id3, 'name':name3, 'grade':grade3})

print('Third user added!')

db.commit()

Third user added!


If you need to insert several users, use executemany and a list with the tuples:


In [10]:
students_ = [(1004, name1, grade1),(1005, name2, grade2),(1006, name3, grade3)]

cursor.executemany('''
    INSERT INTO student(id, name, grade)
                    VALUES(?,?,?)
''', students_)

print('Multiple users added')

db.commit()

Multiple users added


If you need to get the id of the row you just inserted, use lastrowid:

In [13]:
find_id = cursor.lastrowid
print(f'Last row id : {find_id}')

Last row id : 1003


Use rollback to roll back any change to the database since the last call to commit:

In [14]:
cursor.execute('''
    UPDATE student SET grade=? WHERE id=?
''', (65, 1002))

db.rollback()

#### Retrieving Data

To retrieve data, execute a SELECT SQL statement against the cursor object and
then use fetchone() to retrieve a single row or fetchall() to retrieve all the rows.

In [15]:
id_ = 1003

cursor.execute('''
    SELECT name, grade FROM student WHERE id=?
''', (id_,))

student = cursor.fetchone()
print(student)

('Johnny', 68)


The cursor object works as an iterator, invoking fetchall() automatically

In [16]:
cursor.execute('''SELECT name, grade FROM student''')

for row in cursor:
    # row[0] returns the name column values and row[1] returns the grade column values
    print(f'name : {row[0]} grade : {row[1]}')

name : Jimbo grade : 78
name : John grade : 80
name : Johnny grade : 68
name : Jimbo grade : 78
name : John grade : 80
name : Johnny grade : 68
name : Jimbo grade : 78
name : John grade : 80
name : Johnny grade : 68


#### Updating and Deleting Data

Updating or deleting data is practically the same as inserting data:


In [ ]:
grade = 100
id_ = 1001

# update student id 1001
cursor.execute('''
    UPDATE student SET grade=? WHERE id=?
''', (grade,id_))

# Delete student id 1002
id_2 = 1002

cursor.execute('''DELETE FROM student WHERE id=?''', (id_2,))

db.commit()

When we are done working with the DB, we need to close the connection:

In [ ]:
db.close()

#### SQLite Database Exceptions

It is very common for exceptions to occur when working with databases. It is
important to handle these exceptions in your code.
In the example below, we used a try/except/finally clause to catch any exception in
the code. We put the code that we would like to execute but that may throw an
exception (or cause an error) in the try block. Within the except block, we write the
code that will be executed if an exception does occur. If no exception is thrown, the
except block will be ignored. The finally clause will always be executed, whether
an exception was thrown or not. When working with databases, the finally clause
is very important, because it always closes the database connection correctly.

In [ ]:
try:
    # Creates or opens a file called student_db with a SQLite3 DB
    db = sqlite3.connect('student_db')

    # Get a cursor object
    cursor = db.cursor()

    # Check if table users does not exist and create it
    cursor.execute('''CREATE TABLE IF NOT EXISTS
java_programming(id INTEGER PRIMARY KEY, name TEXT,
grade INTEGER)''')
    
    # Commit the change
    db.commit()

# Catch the exception
except Exception as e:

    # Roll back any change if something goes wrong
    db.rollback()
    raise e

finally:
    # Close the db connection
    db.close()